# Ejercicio 1: Introducción a Recuperación de Información

## Objetivo de la práctica
- Entender el problema de **buscar información** en colecciones de texto.
- Comprender por qué se necesita un **índice invertido** en recuperación de información.
- Programar una primera solución manual y luego optimizarla con un índice.
- Evaluar la mejora en tiempos de búsqueda cuando usamos estructuras adecuadas.

## Parte 1: Búsqueda lineal en documentos

### Actividad
1. Se te proporcionará un conjunto de documentos de texto.
2. Escribe una función que:
   - Lea todos los documentos.
   - Busque una palabra ingresada por el usuario.
   - Muestre en qué documentos aparece la palabra.


In [4]:
def buscar_palabra(query):
    archivo = '01_corpus_turismo.txt'
    with open(archivo, 'r') as f:
        lineas = f.readlines()

    encontrado = False
    for i, linea in enumerate(lineas, 1):
        if query in linea:
            print(f"La palabra '{query}' aparece en el documento {i}.")
            encontrado = True

    if not encontrado:
        print(f"La palabra '{query}' no aparece en ningún documento.")

query = input("Ingresa la palabra a buscar: ")

buscar_palabra(query)

Ingresa la palabra a buscar: Ecuador
La palabra 'Ecuador' aparece en el documento 1.


## Parte 2: Construcción de un índice invertido

### Actividad
1. Escribe un programa que:
   - Recorra todos los documentos.
   - Construya un **índice invertido**, es decir, un diccionario donde:
     - Cada palabra clave apunta a una lista de documentos donde aparece.

2. Escribe una nueva función de búsqueda que:
   - Consulte directamente el índice para encontrar los documentos relevantes.
   - Sea mucho más rápida que la búsqueda lineal.

In [13]:
import re
from collections import defaultdict

def construir_indice_invertido(nombre_archivo):
    indice = defaultdict(set)

    with open(nombre_archivo, 'r') as f:
        lineas = f.readlines()

    for num_linea, linea in enumerate(lineas, 1):
        palabras = re.findall(r'\b\w+\b', linea)
        for palabra in palabras:
            indice[palabra].add(num_linea)

    return indice

def buscar_en_indice(indice, palabra):
    if palabra in indice:
        lineas = sorted(indice[palabra])
        print(f"La palabra '{palabra}' aparece en las líneas: {lineas}")
    else:
        print(f"La palabra '{palabra}' no aparece en el texto.")

archivo = '01_corpus_turismo_500.txt'
indice_invertido = construir_indice_invertido(archivo)

query = input("Ingresa la palabra a buscar: ")
buscar_en_indice(indice_invertido, query)


Ingresa la palabra a buscar: Otavalo
La palabra 'Otavalo' aparece en las líneas: [1, 7, 29, 34, 57, 60, 65, 101, 111, 118, 119, 149, 159, 161, 191, 193, 204, 207, 216, 222, 297, 334, 335, 340, 342, 351, 358, 363, 367, 373, 380, 488, 496]


## Parte 3: Evaluación de tiempos de búsqueda
### Actividad

1. Realiza la búsqueda de varias palabras usando:
      -  Corpus pequeño: 16 documentos (turismo en Ecuador).
      -  Corpus grande: 500 documentos (versión ampliada).
2. Mide el tiempo de ejecución:
      -  Para búsqueda lineal.
      -  Para búsqueda usando índice invertido.
      -  Grafica o presenta los resultados en una tabla comparativa.

### Ejemplo de palabras para buscar
- quito
- montañita
- feriado
- playas
- aventura
- galápagos

In [17]:
import re
import time
from collections import defaultdict

def construir_indice_invertido(nombre_archivo):
    indice = defaultdict(set)
    with open(nombre_archivo, 'r') as f:
        lineas = f.readlines()
    for num_linea, linea in enumerate(lineas, 1):
        palabras = re.findall(r'\b\w+\b', linea)
        for palabra in palabras:
            indice[palabra].add(num_linea)
    return indice

def busqueda_lineal(nombre_archivo, palabra):
    with open(nombre_archivo, 'r', encoding='utf-8') as f:
        lineas = f.readlines()
    resultado = []
    for i, linea in enumerate(lineas, 1):
        if palabra in linea:
            resultado.append(i)
    return resultado

def busqueda_indice(indice, palabra):
    return sorted(indice.get(palabra, []))

def evaluar_tiempos(nombre_archivo, palabras):
    print("Evaluación de tiempos:\n")

    t0 = time.time()
    indice = construir_indice_invertido(nombre_archivo)
    t1 = time.time()
    tiempo_indice = t1 - t0

    print(f"Tiempo para construir el índice: {tiempo_indice:.6f} segundos\n")

    print("Palabra\t\tLineal (s)\tÍndice (s)")
    print("-" * 40)

    for palabra in palabras:

        t_start = time.time()
        resultado_lineal = busqueda_lineal(nombre_archivo, palabra)
        t_end = time.time()
        tiempo_lineal = t_end - t_start

        t_start = time.time()
        resultado_indice = busqueda_indice(indice, palabra)
        t_end = time.time()
        tiempo_ii = t_end - t_start

        print(f"{palabra:<12}\t{tiempo_lineal:.6f}\t{tiempo_ii:.6f}")

palabras_prueba = ["quito", "montañita", "feriado", "playas", "aventura", "galápagos"]

archivo = '01_corpus_turismo_500.txt'

evaluar_tiempos(archivo, palabras_prueba)


Evaluación de tiempos:

Tiempo para construir el índice: 0.004142 segundos

Palabra		Lineal (s)	Índice (s)
----------------------------------------
quito       	0.000310	0.000005
montañita   	0.000237	0.000002
feriado     	0.000277	0.000008
playas      	0.000211	0.000007
aventura    	0.000237	0.000006
galápagos   	0.000227	0.000002


## Parte 4:
### Actividad
1. Modifica el índice para que ignore mayúsculas/minúsculas (por ejemplo, "Playa" y "playa" deben considerarse iguales).
2. Permite consultas de múltiples términos (ejemplo: buscar documentos que contengan "playa" y "turismo").
3. Calcula el _speedup_

In [18]:
import re
import time
from collections import defaultdict

def construir_indice_invertido(nombre_archivo):
    indice = defaultdict(set)
    with open(nombre_archivo, 'r', encoding='utf-8') as f:
        lineas = f.readlines()
    for num_linea, linea in enumerate(lineas, 1):
        palabras = re.findall(r'\b\w+\b', linea.lower())
        for palabra in palabras:
            indice[palabra].add(num_linea)
    return indice

def busqueda_lineal(nombre_archivo, consulta):
    terminos = consulta.lower().split()
    with open(nombre_archivo, 'r', encoding='utf-8') as f:
        lineas = f.readlines()

    resultados = set()
    for i, linea in enumerate(lineas, 1):
        linea_l = linea.lower()
        if all(termino in linea_l for termino in terminos):
            resultados.add(i)
    return sorted(resultados)

def busqueda_indice(indice, consulta):
    terminos = consulta.lower().split()
    listas = [indice.get(t, set()) for t in terminos]
    if not listas:
        return []
    resultado = set.intersection(*listas)
    return sorted(resultado)

# Evaluación y Speedup
def evaluar_speedup(nombre_archivo, consultas):
    print("Consulta\t\tLineal(s)\tÍndice(s)\tSpeedup")
    print("-" * 60)

    indice = construir_indice_invertido(nombre_archivo)

    for consulta in consultas:
        t0 = time.time()
        res_lineal = busqueda_lineal(nombre_archivo, consulta)
        t1 = time.time()
        tiempo_lineal = t1 - t0

        t0 = time.time()
        res_indice = busqueda_indice(indice, consulta)
        t1 = time.time()
        tiempo_indice = t1 - t0

        speedup = tiempo_lineal / tiempo_indice if tiempo_indice > 0 else float('inf')

        print(f"{consulta:<20}\t{tiempo_lineal:.6f}\t{tiempo_indice:.6f}\t{speedup:.2f}x")

# PRUEBA
archivo = '01_corpus_turismo.txt'
consultas = [
    "playa",
    "turismo",
    "playa turismo",
    "quito aventura",
    "montañita feriado"
]

evaluar_speedup(archivo, consultas)


Consulta		Lineal(s)	Índice(s)	Speedup
------------------------------------------------------------
playa               	0.000169	0.000006	27.23x
turismo             	0.000070	0.000003	20.93x
playa turismo       	0.000061	0.000004	15.94x
quito aventura      	0.000057	0.000005	12.00x
montañita feriado   	0.000055	0.000004	12.83x
